## importing libraries for use in code

In [564]:

import numpy as np
from sklearn.preprocessing import scale
import pandas as pd
from pandas import DataFrame, Series
import pyodbc
import plotly.plotly as py
import cufflinks as cf
from sklearn.decomposition import RandomizedPCA
from sklearn.cluster import KMeans

## copy of code from CitiesClustering to get the city groupings

In [565]:
array = []
city = []
uber = []
with open(r"C:\Users\Student\Documents\Uber_Project\data\CityStats.csv", 'r') as citystats:
    next(citystats)
    for line in citystats:
        array.append(line.replace("'",'').replace("\n",'').split(','))
        city.append(line.replace("'",'').replace("\n",'').split(',')[0])
        uber.append(line.replace("'",'').replace("\n",'').split(',')[-1])
    

In [566]:
city_np_array = np.array(array)

In [567]:
city_np_array_data= city_np_array.copy()
city_np_array_target= city_np_array.copy()

In [568]:
city_data = city_np_array_data[:,1:-1]

In [569]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(city_np_array_data[:,1:-1])
kmeans.labels_

array([5, 3, 5, 3, 0, 5, 2, 5, 5, 0, 0, 3, 4, 5, 3, 0, 5, 0, 3, 5, 5, 5, 0,
       3, 4, 5, 0, 5, 5, 4, 0, 4, 1, 5, 2, 5, 0, 5, 4, 5, 5, 5, 4, 3, 5, 5,
       0, 0, 3, 4, 3, 0, 0, 0, 5, 0, 5, 5, 5, 5, 5, 3, 5])

In [570]:
u = RandomizedPCA(n_components=2)
data = u.fit_transform(city_data)

C:\Users\Student\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning:

Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.



## creating dataframes of data and plotting with plotly

In [571]:
df = pd.DataFrame(city)  # creating a pandas DataFrame of the list of uk cities

In [572]:
df['labels'] = kmeans.labels_.tolist()  # adding city grouping labels as column in df
df['uber'] = uber  # adding whether or not city has uber to column in df

In [573]:
df.columns=['City', 'Labels','Uber']
g5 = df[df.Labels==5]
g4 = df[df.Labels==4]
g3 = df[df.Labels==3]
g2 = df[df.Labels==2]
g1 = df[df.Labels==1]
g0 = df[df.Labels==0]
# created separate DataFrame from df for each group of cities

In [574]:
c = pyodbc.connect('DSN=sqluberconn;UID=tabi;PWD=password')
# create connection to SQL database with unemployment rate data

In [575]:
unemploy = pd.read_sql_query(sql = r'select [UALAD], [2011], [2012], [2013], [2014], [2015], [2016]from unemployment.[Rates_perc]', con = c)
# run SQL query throurgh connector

In [576]:
g5 = g5.set_index(['City'])
g4 = g4.set_index(['City'])
g3 = g3.set_index(['City'])
g2 = g2.set_index(['City'])
g1 = g1.set_index(['City'])
g0 = g0.set_index(['City'])
# set indexes of group DataFrames to city for ease of joining

In [577]:
unemploy = unemploy.set_index(['UALAD'])
# set indexes of unemployment DataFrame for ease of joining 

In [578]:
g5 = pd.concat([g5, unemploy], axis = 1, join = 'inner')
# join group 5 DataFrame to unemployment 

In [579]:
g5_nuber = g5[g5.Uber=='0']
# create DataFrame of group 5 cities without uber

In [580]:
del g5_nuber['Labels']
del g5_nuber['Uber']
# delete lables and uber column for ease of plotting 

In [581]:
g5_uber = g5[g5.Uber=='1']
del g5_uber['Labels']
del g5_uber['Uber']
# repeat previous process for group 5 cities with uber

In [582]:
g3 = pd.concat([g3, unemploy], axis = 1, join = 'inner')
# join group 3 DataFrame to unemployment 

In [583]:
g3_nuber = g3[g3.Uber=='0']
# create DataFrame of group 3 cities without uber
del g3_nuber['Labels']
del g3_nuber['Uber']
# delete lables and uber column for ease of plotting 

In [584]:
g3_uber = g3[g3.Uber=='1']
del g3_uber['Labels']
del g3_uber['Uber']
# repeat previous process for group 3 cities with uber

In [585]:
g0 = pd.concat([g0, unemploy], axis = 1, join = 'inner')
# join group 0 DataFrame to unemployment 

In [586]:
g0_nuber = g0[g0.Uber=='0']
# create DataFrame of group 0 cities without uber
del g0_nuber['Labels']
del g0_nuber['Uber']
# delete lables and uber column for ease of plotting 

In [587]:
g0_uber = g0[g0.Uber=='1']
del g0_uber['Labels']
del g0_uber['Uber']
# repeat previous process for group 0 cities with uber

In [588]:
us = pd.read_csv(r'C:\Users\Student\Documents\Uber_Project\data\uberstartdates.csv')
# read csv file containing uber start year of each UK city/town where uber operates

In [589]:
us = us.set_index(['City'])
# set index for ease of joining

In [590]:
g5_uber = pd.concat([us, g5_uber], axis = 1, join = 'inner')
# uber start date list to group 5 cities with uber DataFrame

In [591]:
g5_uber14 = g5_uber[g5_uber.UberStart == 2014]
# create DataFrame for cities where uber started in 2014

In [592]:
g5_uber15 = g5_uber[g5_uber.UberStart == 2015]
# create DataFrame for cities where uber started in 2015

In [593]:
del g5_uber14['UberStart']
del g5_uber14['2011']
del g5_uber14['2012']
# delete columns for ease of plotting

In [594]:
del g5_uber15['UberStart']
del g5_uber15['2011']
del g5_uber15['2012']
# delete columns for ease of plotting

In [595]:
g5_uber14 = g5_uber14.apply(pd.to_numeric).mean()
g5_uber15 = g5_uber15.apply(pd.to_numeric).mean()
g5_nuber = g5_nuber.apply(pd.to_numeric).mean()
# change data type of entries to numeric and calculate means

In [596]:
g5_ue = pd.concat([g5_uber14, g5_uber15, g5_nuber], axis = 1, join = 'inner')
# join DataFrames for ease of plotting

In [597]:
g5_ue.columns = ['Uber 2014', 'Uber 2015', 'No Uber']
# name columns of new DataFrame

In [598]:
g5_layout = {'title': 'Group 5 Cities', 'xaxis':{'title': 'Year'}, 'yaxis':{'title': 'Unemployment Rate (%)'}}
# create layout with graph title and axis titles

In [599]:
g5_ue.iplot(kind = 'scatter', layout = g5_layout)
# plot group 5 unemployment rates

In [600]:
g3_uber = pd.concat([us, g3_uber], axis = 1, join = 'inner')
# group 3 had no cities where uber started before 2016 therefore could not be plotted

In [601]:
g0_uber = pd.concat([us, g0_uber], axis = 1, join = 'inner')
# uber start date list to group 0 cities with uber DataFrame

In [602]:
g0_uber14 = g0_uber[g0_uber.UberStart == 2014]
# create DataFrame for cities where uber started in 2014

In [603]:
g0_uber15 = g0_uber[g0_uber.UberStart == 2015]
# create DataFrame for cities where uber started in 2015

In [604]:
del g0_uber14['UberStart']
del g0_uber14['2011']
del g0_uber14['2012']
# delete columns for ease of plotting

In [605]:
del g0_uber15['UberStart']
del g0_uber15['2011']
del g0_uber15['2012']
# delete columns for ease of plotting

In [606]:
g0_uber14 = g0_uber14.apply(pd.to_numeric).mean()
g0_uber15 = g0_uber15.apply(pd.to_numeric).mean()
g0_nuber = g0_nuber.apply(pd.to_numeric).mean()
# change data type of entries to numeric and calculate means

In [607]:
g0_ue = pd.concat([g0_uber14, g0_uber15, g0_nuber], axis = 1, join = 'inner')
# join DataFrames for ease of plotting

In [608]:
g0_ue.columns = ['Uber 2014', 'Uber 2015', 'No Uber']
# name columns of new DataFrame

In [609]:
g0_layout = {'title': 'Group 0 Cities', 'xaxis':{'title': 'Year'}, 'yaxis':{'title': 'Unemployment Rate (%)'}}
# create layout with graph title and axis titles

In [610]:
g0_ue.iplot(kind = 'scatter', layout = g0_layout)
# plot group 5 unemployment rates